<h1>Import Statements<h1>

In [1]:
import tweepy
import snscrape.modules.twitter as sntwitter
import pandas as pd
from nltk.corpus import stopwords
import functools 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diksh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diksh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dtP = pd.read_excel("PoliticalFiguresTwitter.xlsx")
dtP

,Democrat,Republican
0,@RepAdams,@Robert_Aderholt
1,@RepPeteAguilar,@RepMarkAlford
2,@RepColinAllred,@RepRickAllen
3,@RepAuchincloss,@MarkAmodeiNV2
4,@RepBeccaB,@RepArmstrongND
...,...,...
206,@RepDWStweets,@RepWaltzPress
207,@RepMaxineWaters,@TXRandy14
208,@RepBonnie,@RepWebster
209,@RepWexton,@RepBradWenstrup


In [4]:
# Load the stop words list
stop_words = set(stopwords.words('english'))

# Define a function to extract words from a text
def extract_words(text):
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    # Remove non-alphabetic characters
    words = [word for word in words if word.isalpha()]
    return words

In [5]:
def get_tweets(username):
    query = "(from:" +username + ") until:2023-02-18 since:2010-01-01"
    tweets = []
    limit = 100

    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if len(tweets) == limit:
            break
        else:   
            #political or not code         
            tweets.append(extract_words(tweet.content))

    return tweets

<h1>Democrats<h1>

In [6]:
democrats = dtP.loc[:,"Democrat"]
democratTweets = []
for democrat in democrats:
    democratTweets.append(get_tweets(democrat))

C:\Users\diksh\AppData\Local\Temp\ipykernel_18828\309104031.py:11: FutureWarning: content is deprecated, use rawContent instead
  tweets.append(extract_words(tweet.content))


In [7]:
democratTweets = functools.reduce(lambda x, y: x+y, democratTweets) #flattenlist

<h1>Republicans<h1>

In [8]:
republicans = dtP.loc[:,"Republican"]
republicanTweets = []
for republican in republicans:
    republicanTweets.append(get_tweets(republican))

C:\Users\diksh\AppData\Local\Temp\ipykernel_18828\309104031.py:11: FutureWarning: content is deprecated, use rawContent instead
  tweets.append(extract_words(tweet.content))


In [9]:
republicanTweets = functools.reduce(lambda x, y: x+y, republicanTweets) #flattenlist

<h1>Combine

In [10]:
#no need to do this 

#make sure we have equal amount of tweets for merging
# if len(republicanTweets) != len(democratTweets):
#     delta = abs(len(republicanTweets) - len(democratTweets))
#     if len(republicanTweets) > len(democratTweets):
#         republicanTweets = republicanTweets[0:len(republicanTweets) - delta]    
#     else:
#         democratTweets = democratTweets[0:len(democratTweets) - delta]

In [11]:
merge = []
for tweet in democratTweets:
    for word in tweet:
        merge.append([word,0])
for tweet in republicanTweets:
    for word in tweet:
        merge.append([word,1])

In [12]:
tweetsDF = pd.DataFrame(merge,columns=['Tweets','Orientation'])

In [13]:
tweetsDF

,Tweets,Orientation
0,since,0
1,mayor,0
2,vi,0
3,lyles,0
4,instrumental,0
...,...,...
729898,president,1
729899,biden,1
729900,allowed,1
729901,ignore,1


In [15]:
tweetsDF.to_excel("TweetsDataBase.xlsx")  